<figure>
  <IMG SRC="https://raw.githubusercontent.com/fmeer/public-files/main/TUlogo.png" WIDTH=200 ALIGN="right">
</figure>

# Coding the Matrix Method in Python - Notebook 3.1 (In-class examples)
    
In this notebook we discuss the implementation in Python of the theory discussed in Lecture 3 (Q3). We start by defining the classes and performing some sanity checks on simple bars and beams. We then move on to a more complicated frame structure.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## The Node class
The purpose of this class is to store node information and keep track of the total number of DOFs of the problem. Note the automatic bookkeeping we introduce in `__init__`. This simple but efficient way of keeping track of which DOFs belong to which nodes will make life much easier when we need to assemble matrices from multiple elements. **The code below does not need any modification**.

In [2]:
class Node:
    ndof = 0 + 5
    nn   = 0
    
    def clear():
        Node.ndof = 0
        Node.nn = 0
        
    def __init__ (self, x, y):  
        self.x     = x
        self.y     = y
        self.p     = np.zeros(3)

        self.dofs  = [Node.ndof, Node.ndof+1, Node.ndof+2]

        Node.ndof += 3
        Node.nn   += 1

    def add_load (self, p):  
        self.p += p

## The Element class
This class keeps track of each element in the model, including:
- Cross-section properties
- Element orientation (for coordinate system transformations)
- Which Nodes make up each element, and in turn (with help of the Node class) which DOFs belong to each element

Apart from bookkeeping element data, the other main task of this class is to provide the element stiffness matrix in the global coordinate system (for subsequent assembly) and postprocess element-level fields.

Here the class describes an element combining extension and Euler-Bernoulli bending in dynamics. A similar (or inherited) class could also be implemented for different element types (*e.g.* Timoshenko beam, tensioned string, rod in torsion, etc). Here we also keep it simple by assuming elements are all arranged in a 2D plane. 

Please note that, in contrast to statics, the dynamic characterisation of the elements requires the dependency on the excitation frequency which is introduced by the variable "omega". Moreover, all structural systems have damping which is here introduced through the variable "ksi" into the elasticity modulus of the element.

In [3]:
class Element:
    ne = 0

    def clear():
        Element.ne = 0
        
    def __init__ (self, nodes):
        self.nodes = nodes

        self.L = np.sqrt((nodes[1].x - nodes[0].x)**2.0 + (nodes[1].y - nodes[0].y)**2.0)

        dx = nodes[1].x - nodes[0].x
        dy = nodes[1].y - nodes[0].y

        self.cos = dx / self.L
        self.sin = dy / self.L

        R = np.zeros ((6,6))

        R[0,0] = R[1,1] = R[3,3] = R[4,4] = self.cos
        R[0,1] = R[3,4] = -self.sin
        R[1,0] = R[4,3] =  self.sin
        R[2,2] = R[5,5] = 1.0
        
        self.R  = R
        self.Rt = np.transpose(R)

        Element.ne += 1

    def set_section (self, props):
        
        if 'EA' in props:
            self.EA = props['EA']
        else:
            self.EA = 1.e20
            
        if 'ksi' in props: # MODIFIED
            self.ksi = props['ksi'] # MODIFIED
        else: # MODIFIED
            self.ksi = 0.01  # MODIFIED
            
        if 'rhoA' in props:  # MODIFIED
            self.rhoA = props['rhoA']  # MODIFIED
        else:  # MODIFIED
            self.rhoA = 1.e20  # MODIFIED
            
        if 'EI' in props:
            self.EI = props['EI']
        else:
            self.EI = 1.e20
            
        if 'omega' in props:  # MODIFIED
            self.omega = props['omega']  # MODIFIED
        else:   # MODIFIED
            self.omega = 1.e20  # MODIFIED

    def global_dofs  (self):
        return np.hstack ((self.nodes[0].dofs, self.nodes[1].dofs))

    def stiffness ( self ):
        
        k = np.zeros ((6, 6), dtype=complex) # MODIFIED
        
        ksi = self.ksi # MODIFIED
        EA = self.EA * (1 + 2j * ksi) # MODIFIED
        rhoA = self.rhoA  # MODIFIED 
        EI = self.EI * (1 + 2j * ksi) # MODIFIED
        L = self.L
        omega = self.omega  # MODIFIED
        c_r = (EA/rhoA) ** 0.5  # MODIFIED
        beta_r = omega / c_r  # MODIFIED
        beta_b = (omega**2 * rhoA / EI) ** 0.25  # MODIFIED

        # Extension contribution

        k[0,0] = k[3,3] = EA * beta_r * np.cos(beta_r * L) / np.sin(beta_r * L)   # MODIFIED
        k[3,0] = k[0,3] = - EA * beta_r / np.sin(beta_r * L) # MODIFIED

        # Bending contribution
        
        K_beam = np.array([[-EI * beta_b ** 3 * (np.cosh(beta_b * L) * np.sin(beta_b * L) + np.sinh(beta_b * L) * np.cos(beta_b * L)) / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1),EI * np.sinh(beta_b * L) * np.sin(beta_b * L) * beta_b ** 2 / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1),EI * beta_b ** 3 * (np.sinh(beta_b * L) + np.sin(beta_b * L)) / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1),EI * beta_b ** 2 * (np.cosh(beta_b * L) - np.cos(beta_b * L)) / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1)],[EI * np.sinh(beta_b * L) * np.sin(beta_b * L) * beta_b ** 2 / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1),EI * beta_b * (-np.cosh(beta_b * L) * np.sin(beta_b * L) + np.sinh(beta_b * L) * np.cos(beta_b * L)) / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1),EI * beta_b ** 2 * (-np.cosh(beta_b * L) + np.cos(beta_b * L)) / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1),EI * beta_b * (-np.sinh(beta_b * L) + np.sin(beta_b * L)) / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1)],[EI * beta_b ** 3 * (np.sinh(beta_b * L) + np.sin(beta_b * L)) / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1),EI * beta_b ** 2 * (-np.cosh(beta_b * L) + np.cos(beta_b * L)) / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1),-EI * beta_b ** 3 * (np.cosh(beta_b * L) * np.sin(beta_b * L) + np.sinh(beta_b * L) * np.cos(beta_b * L)) / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1),-EI * np.sinh(beta_b * L) * np.sin(beta_b * L) * beta_b ** 2 / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1)],[EI * beta_b ** 2 * (np.cosh(beta_b * L) - np.cos(beta_b * L)) / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1),EI * beta_b * (-np.sinh(beta_b * L) + np.sin(beta_b * L)) / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1),-EI * np.sinh(beta_b * L) * np.sin(beta_b * L) * beta_b ** 2 / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1),EI * beta_b * (-np.cosh(beta_b * L) * np.sin(beta_b * L) + np.sinh(beta_b * L) * np.cos(beta_b * L)) / (np.cosh(beta_b * L) * np.cos(beta_b * L) - 1)]])

        k[1,1] = K_beam[0,0]
        k[1,2] = K_beam[0,1]
        k[1,4] = K_beam[0,2]
        k[1,5] = K_beam[0,3]
        k[2,1] = K_beam[1,0]
        k[2,2] = K_beam[1,1]
        k[2,4] = K_beam[1,2]
        k[2,5] = K_beam[1,3]
        k[4,1] = K_beam[2,0]
        k[4,2] = K_beam[2,1]
        k[4,4] = K_beam[2,2]
        k[4,5] = K_beam[2,3]
        k[5,1] = K_beam[3,0]
        k[5,2] = K_beam[3,1]
        k[5,4] = K_beam[3,2]
        k[5,5] = K_beam[3,3]
        
        #k[1,1] = k[4,4] =  12.0 * EI / L / L / L
        #k[1,4] = k[4,1] = -12.0 * EI / L / L / L
        #k[1,2] = k[2,1] = k[1,5] = k[5,1] = -6.0 * EI / L / L
        #k[2,4] = k[4,2] = k[4,5] = k[5,4] = 6.0 * EI / L / L
        #k[2,2] = k[5,5] = 4.0 * EI / L
        #k[2,5] = k[5,2] = 2.0 * EI / L

        return np.matmul ( np.matmul ( self.Rt, k ), self.R )

## The Constrainer class

This small class keeps track of which DOFs have prescribed displacements and takes care of applying these constraints to the global $\mathbf{K}$ and $\mathbf{f}$. For now we keep it simple and assume all constraints fix the DOF values to zero. For the first two examples you can use this class as it is. For the third assignment, a short new function should be implemented (see below).

In [4]:
class Constrainer:
    def __init__ (self):
        self.dofs = []

    def fix_dof (self, node, dof):
        self.dofs.append (node.dofs[dof])
 
    def fix_node (self, node):
        for dof in node.dofs:
            self.dofs.append (dof)       

    def constrain (self, k, f):
        kc = np.copy (k)
        fc = np.copy (f)
        
        for dof in self.dofs:
            fc[dof] = 0.0
            kc[:,dof] = kc[dof,:] = 0.0
            kc[dof,dof]           = 1.0

        return kc, fc

## Sanity check #1: Bar in extension

Having made our implementations, we now check them with two simple examples that serve as sanity checks. The first is a simple bar undergoing extension:

<center><figure>
  <IMG SRC="https://raw.githubusercontent.com/ibcmrocha/public/main/extpointload.png" WIDTH=200 ALIGN="center">
      </figure></center>

Use the code blocks below to set up and solve this problem using the classes above.

### Parameters

We define the parameters here for convenience. We also use the `clear` functions to restart the node, element and DOF counters. Make sure this is done whenever you start solving a new problem.

In [12]:
EA = 7e6
EI = 1.5 * 7e06 # MODIFIED
rhoA = 1e03  # MODIFIED
F  = 1e06 # MODIFIED
L  = 1
omega = 100  # MODIFIED
ksi = 0.01 # MODIFIED

Node.clear()
Element.clear()

### Create nodes

Create two nodes here. You can store them on a `list` or simply create them as two separate objects (*e.g.* `node1` and `node2`). 

**TIP**: Take a look at which arguments the `__init__` function of the `Node` class receives!

In [13]:
node1 = Node (0,0)
node2 = Node (L,0)

### Create element

Here we only have a single element, so there is no need to store it in a `list` yet. You are also going to need a `dict` defining the cross-section of the element.

**TIP**: See what `__init__` of the `Element` class expects. Also check the `set_section` function.

In [14]:
elem = Element ( [node1, node2] )

section = {}
section['EA'] = EA
section['EI'] = EI
section['ksi'] = ksi  # MODIFIED
section['rhoA'] = rhoA  # MODIFIED
section['omega'] = omega  # MODIFIED

elem.set_section (section)

### Set boundary conditions

We create an instance of the `Constrainer` class to deal with prescribed displacements. Take a look at its functions and inform it Node 1 is fully fixed.

**TIP**: You also need to pass the load $F$ on to Node 2. Check the member functions of `Node` to infer how that should be done.

In [15]:
con = Constrainer()

con.fix_node (node1)
node2.add_load ([F,0,0])

### Assemble the system of equations

Since we only have one element, there is no real assembly to be performed other than getting the stiffness matrix of the single element and storing the load at Node 2 in the correct positions of $\mathbf{f}$. To avoid confusion, we do this part for you. Just take care to change the name of the element or of Node 2 in case you used something different.

In [16]:
global_k = elem.stiffness()
global_f = np.zeros (6)

global_f[3:6] = node2.p 
#print(global_k)
print(global_f)

[      0.       0.       0. 1000000.       0.       0.]


### Constrain the problem and solve for nodal displacements

Try to solve for $u$ as it is written in this block. You will notice that `global_k` is singular. Why is that the case? 

**TIP**: Use the `Constrainer` class to fix that! It already knows three of our DOFs are fixed to zero, you should just let it make the necessary changes to $\mathbf{K}$ and $\mathbf{f}$.

In [17]:
Kc, Fc = con.constrain ( global_k, global_f )
u = np.matmul ( np.linalg.inv(Kc), Fc )
print(u)

[0.        +0.j         0.        +0.j         0.        +0.j
 0.30250885-0.01361971j 0.        +0.j         0.        +0.j        ]


### Compare with the solution you know

Finally, compare the displacement at the end of the bar with the one coming from the ODE solution. Note that $\mathbf{u}$ has six entries!

**TIP**: Remember the DOF ordering we assume ($\left[u\,\,w\,\,\varphi\right]$) and the order we use to create our nodes (check `__init__` of the `Node` class one more time!)

In [18]:
print('ODE solution (0.30250884529318846290, - 0.013619705006987083063 j)','Matrix method solution',u[3]) 
# CHECK MAPLE FILE FOR ANALYTICAL DERIVATIONS

ODE solution (0.30250884529318846290, - 0.013619705006987083063 j) Matrix method solution (0.30250884529318856-0.013619705006987096j)


## Sanity check #2: Cantilever beam

<center><figure>
  <IMG SRC="https://raw.githubusercontent.com/ibcmrocha/public/main/cantilever.png" WIDTH=200 ALIGN="center">
      </figure></center>
    
In the first example above we tested our model under extension. But that does not really guarantee it will behave correctly in bending! That is the goal of this second sanity check. Proceed as before and check with the MAPLE solution:
    
When setting up and solving your model, note that we are now interested in $w$ displacements, our load is now vertical.

### Parameters

In [12]:
EA = 7e6
EI = 1.5 * 7e06 # MODIFIED
rhoA = 1e03  # MODIFIED
F  = 1e06 # MODIFIED
L  = 1
omega = 100  # MODIFIED
ksi = 0.01 # MODIFIED

Node.clear()
Element.clear()

### Create nodes

In [13]:
node1 = Node (0,0)
node2 = Node (L,0)

### Create element

In [14]:
elem = Element ( [node1, node2] )

section = {}
section['EA'] = EA
section['EI'] = EI
section['ksi'] = ksi  # MODIFIED
section['rhoA'] = rhoA  # MODIFIED
section['omega'] = omega  # MODIFIED

elem.set_section (section)

### Set boundary conditions

In [15]:
con = Constrainer()

con.fix_node (node1)
node2.add_load ([0,F,0])

### Assemble the system of equations

In [16]:
global_k = elem.stiffness()
global_f = np.zeros (6)

global_f[0:3] = node1.p
global_f[3:6] = node2.p

### Constrain the problem and solve for nodal displacements

In [17]:
Kc, Fc = con.constrain ( global_k, global_f )
u = np.matmul ( np.linalg.inv(Kc), Fc )
print(u)

[ 0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.0343037 -0.00074182j -0.05114355+0.00109971j]


### Check with the solution you know

In [18]:
print('Analytical solution: (0.0343036985883241-0.000741815967655428j)','Matrix method solution',u[4])

Analytical solution: (0.0343036985883241-0.000741815967655428j) Matrix method solution (0.03430369858832389-0.000741815967655428j)


## Sanity check #3: In-plane frame

<center><figure>
  <IMG SRC="https://raw.githubusercontent.com/ibcmrocha/public/main/vierendeel.png" WIDTH=400 ALIGN="center">
      </figure></center>

In order to solve the problem, you need to improve the classes we define above:
- Right now elements are not rotated to the global system. Implement the correct rotation matrix in the `__init__` function of the `Element` class in order to allow for members with different orientations
- At the moment the `Constrainer` class can only fix all DOFs of a node. Implement a more flexible `fix_dof` function that allows for only certain DOFs to be fixed. Remember our classes assume the DOF order $\left[u\,\, w\,\,\varphi\right]$ that should always be consistent throughout the implementation

Additionally, note that the sanity checks above only had a single element. For this model you need to obtain $\mathbf{K}$ and $\mathbf{f}$ of all elements and add them to the correct locations in the global stiffness matrix and force vector. To do that, make use of the `global_dofs` function of the Element class and the `np.ix_` Numpy utility function. (**TIP**: refer back to the contents of week 2.2 of MUDE!)

You can use the blocks below to develop your answer. Feel free to create new Markdown headers as needed, but otherwise use the ones provided to split your solution into logical parts.

### Parameters

In [19]:
h = 1
b = 1
EIr = 1e6 # MODIFIED
EIk = 1e5 # MODIFIED
EA = 1e10 # MODIFIED
H  = 100
rhoA = 2e03  # MODIFIED
omega = 100  # MODIFIED
ksi = 0.01 # MODIFIED

Node.clear()
Element.clear()

### Create nodes

In [20]:
nodes = []

nodes.append(Node(0,0))
nodes.append(Node(b,0))
nodes.append(Node(b,h))
nodes.append(Node(0,h))

### Create elements

In [21]:
elems = []

elems.append(Element([nodes[0], nodes[1]]))
elems.append(Element([nodes[1], nodes[2]]))
elems.append(Element([nodes[2], nodes[3]]))
elems.append(Element([nodes[0], nodes[3]]))

beams = {}
columns = {}
beams['EI'] = EIr
beams['EA'] = EA
columns['EI'] = EIk
columns['EA'] = EA
beams['ksi'] = ksi  # MODIFIED
beams['rhoA'] = rhoA  # MODIFIED
beams['omega'] = omega  # MODIFIED
columns['ksi'] = ksi  # MODIFIED
columns['rhoA'] = rhoA  # MODIFIED
columns['omega'] = omega  # MODIFIED

elems[0].set_section (beams)
elems[1].set_section (columns)
elems[2].set_section (beams)
elems[3].set_section (columns)

### Set boundary conditions

In [22]:
con = Constrainer()

con.fix_dof (nodes[0], 0)
con.fix_dof (nodes[0], 1)
con.fix_dof (nodes[1], 1)

nodes[3].add_load ([H,0,0])

### Assemble the system

In [23]:
global_k = np.zeros ((3*len(nodes), 3*len(nodes)), dtype=complex) #MODIFIED
global_f = np.zeros (3*len(nodes), dtype=complex)  #MODIFIED

for e in elems:
    elmat = e.stiffness()
    idofs = e.global_dofs()
    
    global_k[np.ix_(idofs,idofs)] += elmat

for n in nodes:
    global_f[n.dofs] += n.p

#print(global_k) 
print(global_f)    

[  0.+0.j   0.+0.j   0.+0.j   0.+0.j   0.+0.j   0.+0.j   0.+0.j   0.+0.j
   0.+0.j 100.+0.j   0.+0.j   0.+0.j]


### Constrain the problem and solve for nodal displacements

In [24]:
Kc, Fc = con.constrain ( global_k, global_f )
u = np.matmul ( np.linalg.inv(Kc), Fc )
print(u)

[ 0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  7.60429805e-07-1.47347670e-08j -1.71208870e-09+4.44787600e-11j
  0.00000000e+00+0.00000000e+00j  7.67790607e-07-1.52380370e-08j
 -2.59531758e-06-1.75236301e-08j  5.36734186e-10-2.34695153e-11j
 -4.68056933e-07+2.09405777e-08j -2.59031516e-06-1.76238584e-08j
 -5.37811961e-10+2.35774627e-11j -4.61819812e-07+2.04465279e-08j]
